In [1]:
import torch
import os
from transformers import TrainerCallback
from trl import SFTTrainer, SFTConfig
from unsloth import FastLanguageModel, is_bf16_supported
from unsloth.chat_templates import get_chat_template  # Import get_chat_template
from datasets import Dataset  # Import Dataset from Hugging Face datasets

class FinetunePhi3_5:
    """
    Text-only fine-tuning of unsloth/Phi-3.5-mini-instruct with LoRA adapters.
    """
    def __init__(
        self,
        data,                           # A list of {"input": str, "output": str} dicts
        epochs=1,
        learning_rate=1e-4,
        warmup_ratio=0.1,
        gradient_accumulation_steps=64,
        optim="adamw_torch",
        model_id="unsloth/Phi-3.5-mini-instruct",
        peft_r=8,
        peft_alpha=16,
        peft_dropout=0.05
    ):
        self.data = data
        self.epochs = epochs
        self.learning_rate = learning_rate
        self.warmup_ratio = warmup_ratio
        self.gradient_accumulation_steps = gradient_accumulation_steps
        self.optim = optim
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model_id = model_id

        # 1. Load base text model and tokenizer
        self.base_model, self.tokenizer = FastLanguageModel.from_pretrained(
            model_name=self.model_id,
            load_in_4bit=False,
            use_gradient_checkpointing=False,
        )

        # 2. Wrap the model with LoRA (text-only)
        self.model = FastLanguageModel.get_peft_model(
            self.base_model,
            target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                            "gate_proj", "up_proj", "down_proj",],
            r=peft_r,
            lora_alpha=peft_alpha,
            lora_dropout=peft_dropout,
            bias="none",
            random_state=3407,
            use_rslora=False,
            loftq_config=None
        )

    def format_data(self, row):
        """
        Format data into the specified template:
        <|user|>
        <|image_1|>{prompt}<|end|>
        <|assistant|>
        {answer}<|end|>
        """
        prompt = row["input"]
        answer = row["output"]
        formatted_text = f"<|user|>\n<|image_1|>{prompt}<|end|>\n<|assistant|>\n{answer}<|end|>"
        return {"text": formatted_text}

    def run(self):
        """
        Execute LoRA fine-tuning on the provided text data.
        """
        # Convert data to the specified format
        formatted_data = [self.format_data(row) for row in self.data]
        dataset = Dataset.from_list(formatted_data)

        # Create SFT config
        training_args = SFTConfig(
            learning_rate=self.learning_rate,
            output_dir='./model_cp',
            optim=self.optim,
            logging_steps=1,
            report_to="none",
            fp16=(not is_bf16_supported()),
            bf16=is_bf16_supported(),
            logging_first_step=True,
            warmup_ratio=self.warmup_ratio,
            per_device_train_batch_size=1,
            per_device_eval_batch_size=1,
            logging_dir='./logs',
            gradient_accumulation_steps=self.gradient_accumulation_steps,
            num_train_epochs=self.epochs,
            weight_decay=0.01,
            lr_scheduler_type="linear",
            seed=3407,
            logging_strategy="steps",
            dataset_kwargs={"skip_prepare_dataset": True},  # Skip Hugging Face's preprocessing
            max_seq_length=2048,
        )

        # Prepare model for training
        FastLanguageModel.for_training(self.model)

        # Initialize and run trainer
        trainer = SFTTrainer(
            model=self.model,
            dataset_text_field="text",
            tokenizer=self.tokenizer,
            train_dataset=dataset,
            args=training_args,
        )
        trainer.train()


# Example usage
if __name__ == "__main__":
    # Sample text-only data
    sample_data = [
        {"input": "Explain gravity", "output": "Gravity is a fundamental force..."},
        {"input": "What is AI?", "output": "AI, or Artificial Intelligence, is..."}
    ]
    finetuner = FinetunePhi3_5(data=sample_data, epochs=1, learning_rate=1e-4)
    finetuner.run()
    print("Fine-tuning complete!")

/home/cqilab/anaconda3/envs/llmfinetune/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.1.6: Fast Llama patching. Transformers: 4.47.1.
   \\   /|    GPU: NVIDIA GeForce RTX 3090. Max memory: 23.684 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 8.6. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post1. FA2 = True]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.07s/it]
Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2025.1.6 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


ValueError: No columns in the dataset match the model's forward method signature. The following columns have been ignored: [text]. Please check the dataset and model. You may need to set `remove_unused_columns=False` in `TrainingArguments`.